In [7]:
import os
import PyPDF2
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

nltk.download("stopwords")
nltk.download("punkt")

# Define the citation patterns (as you did)
citation_patterns = {
    'paranthesis':r'\(\w+ [ve]+ \w+, \d+:\s\d+\)',
    'date_only': r'\(\d{4}\)',
    'APA': r'\(\w+,\s\d{4}\)',
    'APA-Page': r'\(\w+(\s\w+)+,\s\d{4},\s?p\.\s\d+\)',
    'MLA': r'\(\w+\s\d+\)',
    'MLA-Page': r'\(\w+\s\d+,\s?p\.\s\d+\)',
    'Chicago Author-Year': r'\(\w+\s\d{4}\)',
    'Chicago Author-Year-Page': r'\(\w+(\s\w+)+\s\d{4},\s\d+\)',
    'IEEE Numeric': r'\[\d+\]',
    'IEEE-Text': r'\[\d+,\s"[\w\s]+"[,.]?\]',
    'Harvard Author-Year': r'\(\w+\s\d{4}\)',
    'Harvard-Text': r'\(\w+\s\d{4},\s"[\w\s]+"[,.]?\)',
    'Vancouver Numeric': r'\[\d+\]',
    'AMA': r'\[\d+\]',
    'ACS': r'\[\d+\]',
    'CSE': r'\[\d+\]',
    'Bluebook': r'\(\w+\s\d+\)',
    'ASA': r'\(\w+\s\d+\)',
    'AIP': r'\[\d+\]',
    'ISO 690 Numeric': r'\[\d+\]',
    'GSA Style': r'\(\w+\s\d{4}\)',
    'Turabian Style': r'\(\w+\s\d{4}\)',
    'Legal Bluebook': r'\(\w+\s\d+\)',
    'Vancouver Superscript': r'\[\d+\]',
    'Bluebook Law Review': r'\(\w+\s\d+\)',
    'APA Parenthetical': r'\(\w+,\s\d{4}\)',
    'ASA Parenthetical': r'\(\w+\s\d+\)',
    'CMS Author-Date': r'\(\w+\s\d{4}\)',
    'Custom': r'\w+(\sve\s\w+)?\s\(\d{4}:\s\d+-\d+\)',
    'Legal Citations with Statutes': r'\(\w+\s\d+\)',
    'Legal Citations with Court Cases': r'\(\w+\s\d+\)',
    'ISO 690-2 Numeric': r'\[\d+\]',
    'ISO 690-2 Author-Date': r'\(\w+\s\d{4}\)',
    #'APA Parenthetical': r'\(\w+,\s\d{4}\)',
    'ASA Parenthetical': r'\(\w+\s\d+\)',
    'CMS Author-Date': r'\(\w+\s\d{4}\)',
    'Legal Citations with Statutes': r'\(\w+\s\d+\)',
    'Legal Citations with Court Cases': r'\(\w+\s\d+\)',
    'ISO 690-2 Numeric': r'\[\d+\]',
    'ISO 690-2 Author-Date': r'\(\w+\s\d{4}\)',
    'APA (American Psychological Association)': r'\(\w+\s&\s\w+,\s\d{4}\)',
    'APA-Three or More Authors': r'\(\w+\set\sal\.,\s\d{4}\)',
    'APA-Group Author with Abbreviation': r'\(World Health Organization\s\[WHO\],\s\d{4}\)',
    'APA-Direct Quote': r'\(\w+’s\sLast\sName,\s\d{4},\sp\.\sPage\sNumber\)',
    'APA-Secondary Source': r'\(Primary\sAuthor’s\sLast\sName,\sas\scited\sin\sSecondary\sAuthor’s\sLast\sName,\s\d{4}\)',
    'APA-Multiple Works by the Same Author in the Same Year': r'\(\w+’s\sLast\sName,\s\d{4}a\)',
    'APA-Personal Communication': r'\(P\.\s\w+,\spersonal\scommunication,\s\w+\s\d{1,2},\s\d{4}\)',
    'APA-Citing a Source Within a Source': r'\(\w+’s\sLast\sName,\sas\scited\sin\sSecondary\sAuthor’s\sLast\sName,\s\d{4}\)'
}

# Combine the regex patterns into a list (as you did)
regex_patterns = list(citation_patterns.values())

class Pdf2Text(object):

    def __int__(self):
        pass

    def pdf2txt(self, pdf_path):
        text = ""

        # Open the PDF file in read-binary mode
        with open(pdf_path, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)

            # Iterate through each page in the PDF
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text += page.extract_text()

        return text

    def pdfs2txt(self, directory_path):
        # Initialize an empty list to store the text from each PDF
        text_list = []

        # Iterate through the files in the directory
        for filename in os.listdir(directory_path):
            if filename.endswith(".pdf"):
                # Construct the full path to the PDF file
                pdf_file_path = os.path.join(directory_path, filename)

                # Open the PDF file in read-binary mode
                with open(pdf_file_path, 'rb') as pdf_file:
                    pdf_reader = PyPDF2.PdfReader(pdf_file)

                    # Initialize text for the current PDF
                    pdf_text = ""

                    # Iterate through each page in the PDF
                    for page_num in range(len(pdf_reader.pages)):
                        page = pdf_reader.pages[page_num]
                        pdf_text += page.extract_text()

                    # Append the extracted text to the list
                    text_list.append(pdf_text)

        return text_list


class CitationExtractor(object):

    def __int__(self):
        pass

    def extractCitationSingleDocument(self, text, style):

        pairs = {
            'paranthesis': [],
            'date_only': [],
            'APA': [],
            'APA-Page': [],
            'MLA': [],
            'MLA-Page':[],
            'Chicago Author-Year':[],
            'Chicago Author-Year-Page': [],
            'IEEE Numeric': [],
            'IEEE-Text': [],
            'Harvard Author-Year': [],
            'Harvard-Text': [],
            'Vancouver Numeric': [],
            'AMA': [],
            'ACS': [],
            'CSE': [],
            'Bluebook': [],
            'ASA': [],
            'AIP': [],
            'ISO 690 Numeric': [],
            'GSA Style': [],
            'Turabian Style': [],
            'Legal Bluebook': [],
            'Vancouver Superscript': [],
            'Bluebook Law Review': [],
            #'APA Parenthetical': [],
            'ASA Parenthetical': [],
            'CMS Author-Date': [],
            'Custom': [],
            'Legal Citations with Statutes':[],
            'Legal Citations with Court Cases': [],
            'ISO 690-2 Numeric': [],
            'ISO 690-2 Author-Date': [],
            'APA Parenthetical': [],
            'ASA Parenthetical': [],
            'CMS Author-Date': [],
            'Legal Citations with Statutes': [],
            'Legal Citations with Court Cases': [],
            'ISO 690-2 Numeric':[],
            'ISO 690-2 Author-Date': [],
            'APA (American Psychological Association)': r'\(\w+\s&\s\w+,\s\d{4}\)',
            'APA-Three or More Authors': [],
            'APA-Group Author with Abbreviation': [],
            'APA-Direct Quote': [],
            'APA-Secondary Source': [],
            'APA-Multiple Works by the Same Author in the Same Year': [],
            'APA-Personal Communication': [],
            'APA-Citing a Source Within a Source': []
        }

        sentences = re.split(r'(?<=[.!?])\s+', text)

        if style == "all":
            for sentence in sentences:
                for pattern_name, pattern in citation_patterns.items():
                    matches = re.findall(pattern, sentence)
                    if matches:
                        pairs[pattern_name].append(sentence)

            return pairs

        for sentence in sentences:
                matches = re.findall(citation_patterns[style], sentence)
                if matches:
                    pairs[style].append(sentence)

        return pairs


pdf2txt = Pdf2Text()
extractor = CitationExtractor()

# Read the text from a PDF file
path = "10.25064-mulkiye.1050139-2163743.pdf"
txt = pdf2txt.pdf2txt(path)

# Split the extracted text into sentences
sentences = re.split(r'(?<=[.!?])\s+', txt)

# Extract references from the text
intext = extractor.extractCitationSingleDocument(txt, "all")
intext = [v for v in intext.values() if len(v) != 0]
intext_flattened_list = [item for sublist in intext for item in sublist]

# Read the references part from the PDF (kaynakca)
kaynakca = str(txt.split("Kaynakça")[1])
kaynakca = kaynakca.split("\n")

# Define a function to calculate Jaccard similarity between two sentences
def jaccard_similarity(sentence1, sentence2):
    set1 = set(word_tokenize(sentence1))
    set2 = set(word_tokenize(sentence2))
    intersection = len(set1.intersection(set2))
    union = len(set1) + len(set2) - intersection
    return intersection / union

# Initialize a list to store matched references
matched_references = []

# Iterate through references and sentences to find matches using Jaccard similarity
for ref in kaynakca:
    max_similarity = 0
    best_match = None
    for sentence in intext_flattened_list:
        similarity = jaccard_similarity(ref, sentence)
        if similarity > max_similarity:
            max_similarity = similarity
            best_match = sentence
    matched_references.append((ref, best_match, max_similarity))

# Sort matches by Jaccard similarity
matched_references.sort(key=lambda x: x[2], reverse=True)

# Print the matching references
for match in matched_references:
    reference_ref = match[0]
    reference = match[1]
    similarity = match[2]
    print("Reference Ref:", reference_ref)
    print("Matched Reference:", reference)
    print("Jaccard Similarity:", similarity)
    print("--------------------------------------------------------")


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nilakay/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /Users/nilakay/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Reference Ref: Yanık C ve Öztürk M (2014). Toplumsal Hareketlerin Dönüşümü Üzerine Bir 
Matched Reference: Yanık C ve Öztürk M (2014).
Jaccard Similarity: 0.6428571428571429
--------------------------------------------------------
Reference Ref: Tandoğan Ş M (2021) Yeni Paradigma Bağlamında Ortadoğu’daki Toplumsal Hareketlerin 
Matched Reference: 587
Tandoğan Ş M (2021) Yeni Paradigma Bağlamında Ortadoğu’daki Toplumsal Hareketlerin 
İncelenmesi: Mısır ve İran Örnekleri.
Jaccard Similarity: 0.6363636363636364
--------------------------------------------------------
Reference Ref: Tandoğan Ş M (2021) Yeni Paradigma Bağlamında Ortadoğu’daki Toplumsal Hareketlerin 
Matched Reference: 587
Tandoğan Ş M (2021) Yeni Paradigma Bağlamında Ortadoğu’daki Toplumsal Hareketlerin 
İncelenmesi: Mısır ve İran Örnekleri.
Jaccard Similarity: 0.6363636363636364
--------------------------------------------------------
Reference Ref: Tandoğan Ş M (2021) Yeni Paradigma Bağlamında Ortadoğu’daki Toplumsal Hare

In [8]:
len(matched_references)

128